# Artificial Neural Network and Deep Learning 
## Challenge 1: Image Classification
### Team:   Bergamasco Alex 10521973

This is the first approach that i used in this challenge.
I started building a custom CNN performing parameters tuning both in the data augmentation section and in the hyperparameters of the neural network.

At the end, I reached a 0.70 accuracy with this model.


After this approach, I continued the challenge performing Transfer Learning with different pre-trained models (explained in the other notebook)


### Data Augmentation

With some trial and error approach I tuned parameters in the ImageDataGenerator. These are the parameters that I used in my best model.


### Creating the CNN

I used the same code-structure of the lectures, I found it very useful, principally when I tried to add some dropout and batch normalization layers between blocks.

List of actions:
- I tried to add some layers but I didn't reach any improvement.
- Modify parameters in data augmentation with (a lot of) trial & error technique. Reached 0.56-0.6 accuracy
- Changed the image_size (300x300 performs good w.r.t. 256x256) because of the shape of the original images.
- Added dropout and batch normalization layers to avoid overfitting and it performed very well, getting a 0.7 of accuracy.

There are some intermediate steps that I don't mentioned, related principally to hyperparameters tuning.

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

SEED = 1234

tf.random.set_seed(SEED)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

cwd = os.getcwd()

### Creating a validation set

In [ ]:
import split_folders

# Split with a ratio of .8 and .2 (train and validation)
split_folders.ratio('/home/alex/Desktop/Keras_Lectures/competition/Classification_Dataset/training', output="output", seed=1337, ratio=(.8, .2)) # default values

### Performing Data Augmentation

In [ ]:
#Set true to perform data augmentation
data_aug = True

if data_aug:
    train_data_gen = ImageDataGenerator(rotation_range=30,
                                        width_shift_range=0.1,
                                        height_shift_range=0.1,
                                        shear_range=0.01,
                                        zoom_range=[0.9, 1.1],
                                        horizontal_flip=True,
                                        vertical_flip=False,
                                        fill_mode='reflect',
                                        brightness_range=[0.5, 1.5],
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

### Flow_from_directory

In [ ]:
dataset_dir = os.path.join(cwd, 'Classification_Dataset')

bs = 8

# Image Shape
img_h = 300
img_w = 300

num_classes=20

input_shape = (img_h, img_w, 3)

decide_class_indices = True

if decide_class_indices:
    classes = [ 'owl',
                'galaxy',
                'lightning',
                'wine-bottle',
                't-shirt',
                'waterfall',
                'sword',
                'school-bus',
                'calculator',
                'sheet-music',
                'airplanes',
                'lightbulb',
                'skyscraper',
                'mountain-bike',
                'fireworks',
                'computer-monitor',
                'bear',
                'grand-piano',
                'kangaroo',
                'laptop']  
else:
    classes=None


training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs,
                                               classes=classes,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  


validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=bs, 
                                               classes=classes,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

### Creating training and validation dataset from the generator

In [ ]:
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

valid_dataset = valid_dataset.repeat()

### Creating the CNN

I used the same code-structure of the lectures, I found it very useful, principally when I tried to add some dropout and batch normalization layers between blocks.

List of actions:
- I tried to add some layers but I didn't reach any improvement.
- Modify parameters in data augmentation with (a lot of) trial & error technique. Reached 0.56-0.6 accuracy
- Changed the image_size (300x300 performs good w.r.t. 256x256) because of the shape of the original images.
- Added dropout and batch normalization layers to avoid overfitting and it performed very well, getting a 0.7 of accuracy.

There are some intermediate steps that I don't mentioned, related principally to hyperparameters tuning.

In [ ]:
class ConvBlock(tf.keras.Model):
    def __init__(self, num_filters):
        super(ConvBlock, self).__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters=num_filters,
                                             kernel_size=(3, 3),
                                             strides=(1, 1), 
                                             padding='same')
        self.activation = tf.keras.layers.ReLU()
        self.batchnormalization = tf.keras.layers.BatchNormalization()
        self.pooling = tf.keras.layers.MaxPool2D(pool_size=(2, 2))
        self.dropout = tf.keras.layers.Dropout(0.2) 
        
    def call(self, inputs):
        x = self.conv2d(inputs)
        x = self.activation(x)
        x = self.batchnormalization(x)
        x = self.pooling(x)
        x = self.dropout(x)
        return x

In [ ]:
depth = 5
start_f = 8
num_classes = 20

class CNNClassifier(tf.keras.Model):
    def __init__(self, depth, start_f, num_classes):
        super(CNNClassifier, self).__init__()
        
        self.feature_extractor = tf.keras.Sequential()
    
        for i in range(depth):
            self.feature_extractor.add(ConvBlock(num_filters=start_f))
            start_f *= 2
            
        self.flatten = tf.keras.layers.Flatten()
        self.classifier = tf.keras.Sequential()
        self.classifier.add(tf.keras.layers.Dense(units=512, activation='relu', kernel_initializer='he_uniform'))
        self.classifier.add(tf.keras.layers.BatchNormalization())
        self.classifier.add(tf.keras.layers.Dropout(0.4)) 
        self.classifier.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
        
    def call(self, inputs):
        x = self.feature_extractor(inputs)
        x = self.flatten(x)
        x = self.classifier(x)
        return x

model = CNNClassifier(depth=depth,
                      start_f=start_f,
                      num_classes=num_classes)

model.build(input_shape=(None, img_h, img_w, 3))

In [ ]:
model.feature_extractor.summary()
model.summary()
model.weights

### Optimization parameters

In [ ]:
loss = tf.keras.losses.CategoricalCrossentropy()

lr = 1e-3

optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

metrics = ['accuracy']

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
model.fit(x=train_dataset,
          epochs=60,
          steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen))

### Get predictions on the test set

In [ ]:
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
from PIL import Image
test_dir = os.path.join(cwd, 'Classification_Dataset/test')
image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:
    img = Image.open(os.path.join(test_dir,'{}').format(image_name)).convert('RGB')
    img = img.resize((img_h, img_w))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, 0) 
    
    out_softmax = model.predict(x=img_array / 255.)
    
    predicted_class = np.argmax(out_softmax, -1)
    
    predicted_class = predicted_class[0]
    
    results[image_name] = predicted_class
 
create_csv(results)